# Model Router

**Model router** for Azure AI Foundry is a deployable AI chat model that is trained to select the best large language model (LLM) to respond to a given prompt in real time. By evaluating factors like query complexity, cost, and performance, it intelligently routes requests to the most suitable model. Thus, it delivers high performance while saving on compute costs where possible, all packaged as a single model deployment.

**Model router intelligently selects the best underlying model for a given prompt to optimize costs** while maintaining quality. Smaller and cheaper models are used when they're sufficient for the task, but larger and more expensive models are available for more complex tasks. Also, reasoning models are available for tasks that require complex reasoning, and non-reasoning models are used otherwise. Model router provides a single deployment and chat experience that combines the best features from all of the underlying chat models.

Model router will use these models:
- 2025-05-19	GPT-4.1 (2025-04-14)
- GPT-4.1-mini (2025-04-14)
- GPT-4.1-nano (2025-04-14)
- o4-mini (2025-04-16)

In [1]:
import base64
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

endpoint = os.getenv("azure_openai_endpoint")
model = "model-router"

In [3]:
# Initialize Azure OpenAI client with Entra ID authentication
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2025-01-01-preview",
)

In [4]:
def modelrouter(prompt):
    """
    Generates a response from the chat model based on the provided prompt.

    Args:
        prompt (str): The user's input to which the chat model will respond.

    Returns:
        response: The complete response object from the chat model, including the generated message and model details.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            temperature=0.7,
        )

        print("\033[1;31;34m")
        print(response.choices[0].message.content)
        print("\033[1;31;32m")
        print(f"This model was automatically selected: {response.model}")
        return response

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


## Examples

### Basic prompt

In [5]:
prompt = "What is FIFA?"

modelrouter(prompt)


FIFA, which stands for the Fédération Internationale de Football Association, is the international governing body of association football (soccer). Founded in 1904 and headquartered in Zurich, Switzerland, FIFA is responsible for overseeing the global organization and regulation of the sport. Its duties include organizing international competitions such as the FIFA World Cup, establishing the rules of the game, and promoting the development of football worldwide. FIFA also works to ensure fair play, maintain the integrity of the sport, and support national football associations across its member countries.

This model was automatically selected: gpt-4.1-nano-2025-04-14


ChatCompletion(id='chatcmpl-BduTkM0iyB1HUTPss7R2RKU9GLqeA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='FIFA, which stands for the Fédération Internationale de Football Association, is the international governing body of association football (soccer). Founded in 1904 and headquartered in Zurich, Switzerland, FIFA is responsible for overseeing the global organization and regulation of the sport. Its duties include organizing international competitions such as the FIFA World Cup, establishing the rules of the game, and promoting the development of football worldwide. FIFA also works to ensure fair play, maintain the integrity of the sport, and support national football associations across its member countries.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filt

> gpt-4.1-nano was chosen

### Medium complex prompt

In [6]:
prompt = "Who  won the last world cup? Explain the details for the country in terms of financial aspects."

modelrouter(prompt)


The most recent FIFA World Cup (as of June 2024) was held in Qatar in 2022. The winner of the 2022 FIFA World Cup was **Argentina**. They defeated France in a dramatic final, which ended 3-3 after extra time, with Argentina winning 4-2 on penalties.

**Financial Aspects for Argentina:**

1. **Prize Money:**
   - Argentina received $42 million in prize money from FIFA for winning the tournament, which is the highest amount ever awarded to a World Cup winner.
   - This money is usually distributed among the football federation, players, coaching staff, and for investment in football development.

2. **Boost to Football Economy:**
   - The victory significantly increased the value of Argentine football, both domestically and internationally.
   - Merchandise sales (jerseys, memorabilia, etc.) saw a huge boost, with millions of shirts sold worldwide, especially those with Lionel Messi’s name and number.

3. **Sponsorship and Endorsements:**
   - Major brands and sponsors increased their a

ChatCompletion(id='chatcmpl-BduTl2BCoUZajwqdYtu3E1U0s5vps', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The most recent FIFA World Cup (as of June 2024) was held in Qatar in 2022. The winner of the 2022 FIFA World Cup was **Argentina**. They defeated France in a dramatic final, which ended 3-3 after extra time, with Argentina winning 4-2 on penalties.\n\n**Financial Aspects for Argentina:**\n\n1. **Prize Money:**\n   - Argentina received $42 million in prize money from FIFA for winning the tournament, which is the highest amount ever awarded to a World Cup winner.\n   - This money is usually distributed among the football federation, players, coaching staff, and for investment in football development.\n\n2. **Boost to Football Economy:**\n   - The victory significantly increased the value of Argentine football, both domestically and internationally.\n   - Merchandise sales (jerseys, memorabilia, etc.) saw a huge boost, with milli

> gpt-4.1 was chosen

### Reasoning prompt

In [7]:
prompt = """
Who won the 2022 FIFA World Cup? Consider different perspectives, analyze the impact of the victory on global football, and discuss the team’s tactical approach in key matches.
"""

modelrouter(prompt)


The 2022 FIFA World Cup was won by Argentina, who beat France 4–2 on penalties after a 3–3 draw in the final in Doha on December 18, 2022.  Below is a multi-layered look at that victory—from different perspectives, its ripple effects across world football, and the tactical blueprint Argentina deployed in key matches.

1. Different Perspectives  
  • Argentinian fans and media  
    – A national catharsis and celebration of Lionel Messi, who had never lifted football’s greatest prize despite a glittering career.  
    – A unifying moment amid ongoing economic and political challenges at home.  
  • Global neutrals  
    – A classic final that many neutrals hailed as one of the most dramatic in modern World Cup history.  
    – A last-hurrah for Messi on football’s biggest stage, elevating the final to near-mythic status.  
  • French perspective  
    – Despite battling back from 2–0 down to level at 2–2 (and later 3–3), France ultimately fell short on penalties—prompting soul-searchin

ChatCompletion(id='chatcmpl-BduTsxBazf1QFpwcfBl97sq9ylAhz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2022 FIFA World Cup was won by Argentina, who beat France 4–2 on penalties after a 3–3 draw in the final in Doha on December 18, 2022.  Below is a multi-layered look at that victory—from different perspectives, its ripple effects across world football, and the tactical blueprint Argentina deployed in key matches.\n\n1. Different Perspectives  \n  • Argentinian fans and media  \n    – A national catharsis and celebration of Lionel Messi, who had never lifted football’s greatest prize despite a glittering career.  \n    – A unifying moment amid ongoing economic and political challenges at home.  \n  • Global neutrals  \n    – A classic final that many neutrals hailed as one of the most dramatic in modern World Cup history.  \n    – A last-hurrah for Messi on football’s biggest stage, elevating the final to near-mythic status.

> o4 mini was chosen